In [2]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth
import random

In [3]:
# Initialize Spotify client
client_credentials_manager = SpotifyClientCredentials(client_id='4124d1d1c0b547c889b174b29cc43af7', client_secret='74bb1bd9b862440ea797bd1a671cd9ac')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

def get_spotify_track_id(song_name, artist_name):
    # Search for tracks
    results = sp.search(q=f"track:{song_name} artist:{artist_name}", type='track', limit=1)
    
    # Extract track ID if available
    if results['tracks']['items']:
        track_id = results['tracks']['items'][0]['id']
        return track_id
    else:
        return None

In [4]:
# read in song_moods.csv + process the data
# spotify id?
# {mood: songs}
def read_songs(file_path):
    df = pd.read_csv(file_path)
    
    mood_names = df.columns.values.tolist()[2:]
    
    mood_songs = {}
    for i, row in df.iterrows():
        title, artist, moods = row[0], row[1], row[2:]
        spotify_uri = get_spotify_track_id(title, artist)
        
        for mood, val in enumerate(moods):
            if val == 1:
                mood_name = mood_names[mood]
                mood_songs.setdefault(mood_name, []).append({'spotify_uri': spotify_uri, 'title': title, 'artist': artist})
                
    return mood_songs

In [5]:
moods = read_songs('song_moods.csv')

/var/tmp/ipykernel_25285/1874800133.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  title, artist, moods = row[0], row[1], row[2:]


In [6]:
len(moods)

172

In [7]:
# given moods and moods dict
# spotify id?
# return songs based on those moods
def extract_random_songs(mood_dict, moods, num_songs_per_mood=3):
    random_songs = set()

    for mood in moods:
        if mood in mood_dict:
            songs = mood_dict[mood]
            if len(songs) > num_songs_per_mood:
                # random_songs.update(random.sample(songs, min(num_songs_per_mood, len(songs))))
                sampled_songs = random.sample(songs, min(num_songs_per_mood, len(songs)))
                random_songs.update(song['spotify_uri'] for song in sampled_songs if song['spotify_uri'] is not None)
            else:
                random_songs.update(song['spotify_uri'] for song in songs if song['spotify_uri'] is not None)
        else:
            print(f"No songs found for mood: {mood}")

    return random_songs

In [8]:
# given list of songs
# create spotify playlist
# get spotify to recommend songs based on those songs
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id='4124d1d1c0b547c889b174b29cc43af7',
                                               client_secret='74bb1bd9b862440ea797bd1a671cd9ac',
                                               redirect_uri='http://localhost:8888/callback',
                                               scope='playlist-modify-private'))

def create_spotify_playlist(playlist_name, track_uris):
    # random_track_uri = random.sample(track_uris, min(5, len(track_uris)))
    playlist = sp.user_playlist_create(sp.me()['id'], playlist_name, public=False)
    # recommendations = sp.recommendations(seed_tracks=random_track_uri, limit=30)
    # recommended_track_uris = [track['uri'] for track in recommendations['tracks']]
    sp.playlist_add_items(playlist['id'], track_uris)
    
    return playlist['id']

In [9]:
songs = extract_random_songs(moods, ['angry', 'angst-ridden', 'brooding', 'confident'])

In [10]:
songs

{'03yH8hIyiNOMVnR9HcQeZx',
 '0ASYYaJ8RMrZ7N44KMEJr8',
 '1UZbE63mMVUe5ql4fk1UjT',
 '1klrDyGRsCBPofZdqdaZp6',
 '22bPsP2jCgbLUvh82U0Z3M',
 '2QcgSxSdGmiCk4G7FOC5N9',
 '3TEAWiw3w0IAsoj4p3tJ7x',
 '6GnQ9TEKUjqGnLqmTTpNS2',
 '6UmiroOo97nxwJTYjWBmKE'}

In [11]:
create_spotify_playlist("testing!!! acv", songs)

KeyboardInterrupt: 

In [ ]:
# stretch: personalize based on user listening behavior